In [1]:
# !pip install --upgrade tensorflow --quiet
# !pip install keras_tuner --quiet
# !pip install tensorflow-io --quiet
# # Google colab modules
# from google.colab import drive
import sys, importlib

# # Mount drive
# drive.mount('/content/gdrive', force_remount=True)
ROOT_PATH = './'
# sys.path.append(ROOT_PATH)

import coremlv2 as core
core._init_ml()
# core._init_models()
core.os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Reload coreml
importlib.reload(core)
import keras_tuner as kt

#### Use multiple GPUs

In [2]:
memory_limit = 2816 # 2 + 4
memory_limit = 2300 # 3 + 5
memory_limit = 3500 # 2 + 3
memory_limit = 1700 # 4 + 6
memory_limit = 1850 # 4 + 6, display moved to titan X
memory_limit = 1750 # 4 + 6, display moved to titan X, increase gpu free memory
gpus = core.tf.config.list_physical_devices('GPU')
if gpus:
    # Create virtual GPUs
    try:
        core.tf.config.set_logical_device_configuration(
            gpus[0],
            [core.tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit),
             core.tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit),
             core.tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit),
             core.tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit)])
        core.tf.config.set_logical_device_configuration(
            gpus[1],
            [core.tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit),
             core.tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit),
             core.tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit),
             core.tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit),
             core.tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit),
             core.tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit)])
        logical_gpus = core.tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs, ", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
    

2 Physical GPUs,  10 Logical GPUs


In [2]:
memory_limit = 1750 # 6 vgpu
memory_limit = 3500 # 3 vgpu
memory_limit = 5250 # 2 vgpu
# memory_limit = 3000 # 2 vgpu
gpus = core.tf.config.list_physical_devices('GPU')
if gpus:
    # Create virtual GPUs
    try:
        core.tf.config.set_logical_device_configuration(
            gpus[0],
            [core.tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit),
             core.tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit)])
        logical_gpus = core.tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs, ", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs,  2 Logical GPUs


In [2]:
# Limiting GPU memory growth
gpus = core.tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            core.tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = core.tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs, ", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs,  1 Logical GPUs


### Manually retrain model

In [3]:
model_base_id = '327'
version = '3'
model_no = model_base_id
constituent_limits = 0.025
id_constituent = 1
min_vid_constituents = 0.5
epochs = 5
iter_id = f'model-{model_no}-{version}_constituent_limits-{constituent_limits}_id_constituent-{id_constituent}_min_vid_constituents-{min_vid_constituents}_epochs-{epochs}'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'
batch_size = 256
shuffle_buffer_size = 1
generator=True

In [4]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

history.history['batch_size'] = batch_size
history.history['shuffle_buffer_size'] = shuffle_buffer_size

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 1149
Total constituents: 422
Epoch 1/5
4879/4879 [==============================] - 1353s 276ms/step - loss: 0.6750 - accuracy: 0.5869 - val_loss: 0.6662 - val_accuracy: 0.5994
Epoch 2/5
4879/4879 [==============================] - 1445s 296ms/step - loss: 0.6591 - accuracy: 0.6071 - val_loss: 0.6657 - val_accuracy: 0.6018
Epoch 3/5
4879/4879 [==============================] - 1576s 323ms/step - loss: 0.6538 - accuracy: 0.6131 - val_loss: 0.6776 - val_accuracy: 0.5756
Epoch 4/5
4879/4879 [==============================] - 1465s 300ms/step - loss: 0.6478 - accuracy: 0.6190 - val_loss: 0.7006 - val_accuracy: 0.5678
Epoch 5/5
4879/4879 [==============================] - 1513s 310ms/step - loss: 0.6415 - accuracy: 0.6237 - val_loss: 0.7225 - val_accuracy: 0.5590


In [4]:
model_base_id = '327'
version = '3'
model_no = model_base_id
constituent_limits = 0.025
id_constituent = 1
min_vid_constituents = 0.5
epochs = 10
iter_id = f'model-{model_no}-{version}_constituent_limits-{constituent_limits}_id_constituent-{id_constituent}_min_vid_constituents-{min_vid_constituents}_epochs-{epochs}'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'
batch_size = 4096
shuffle_buffer_size = 1
generator=True

In [5]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

history.history['batch_size'] = batch_size
history.history['shuffle_buffer_size'] = shuffle_buffer_size

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 1149
Total constituents: 422
Epoch 1/10


InternalError:    Failed to call ThenRnnForward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 256, 256, 1, 120, 4096, 256] 
	 [[{{node CudnnRNN}}]]
	 [[model/lstm_1/PartitionedCall]] [Op:__inference_train_function_431164]

Function call stack:
train_function -> train_function -> train_function


In [3]:
model_base_id = '327'
version = '4'
model_no = model_base_id
constituent_limits = 0.025
id_constituent = 1
min_vid_constituents = 0.5
epochs = 10
batch_size = 1024
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [4]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 1149
Total constituents: 422
Epoch 1/10
1220/1220 [==============================] - 1437s 1s/step - loss: 0.6906 - accuracy: 0.5579 - val_loss: 0.6789 - val_accuracy: 0.5665
Epoch 2/10
1220/1220 [==============================] - 1392s 1s/step - loss: 0.6600 - accuracy: 0.6013 - val_loss: 0.6691 - val_accuracy: 0.5814
Epoch 3/10
1220/1220 [==============================] - 1359s 1s/step - loss: 0.6536 - accuracy: 0.6107 - val_loss: 0.6691 - val_accuracy: 0.5839
Epoch 4/10
1220/1220 [==============================] - 1348s 1s/step - loss: 0.6494 - accuracy: 0.6160 - val_loss: 0.6676 - val_accuracy: 0.5865
Epoch 5/10
1220/1220 [==============================] - 1352s 1s/step - loss: 0.6457 - accuracy: 0.6201 - val_loss: 0.6704 - val_accuracy: 0.5844
Epoch 6/10
1220/1220 [==============================] - 1356s 1s/step - loss: 0.6416 - accuracy: 0.6242 - val_loss: 0.6705 - val_accuracy: 0.5853
Epoch 7/10
1220/1220 [==============================] - 1393s 1s/step - los

In [3]:
model_base_id = '327'
version = '3'
model_no = model_base_id
constituent_limits = 0
id_constituent = 1
min_vid_constituents = 0.5
epochs = 10
batch_size = 1024
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [4]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 422
Total constituents: 422
Epoch 1/10
825/825 [==============================] - 690s 828ms/step - loss: 0.6952 - accuracy: 0.5652 - val_loss: 0.6669 - val_accuracy: 0.5835
Epoch 2/10
825/825 [==============================] - 677s 821ms/step - loss: 0.6607 - accuracy: 0.6000 - val_loss: 0.6629 - val_accuracy: 0.5888
Epoch 3/10
825/825 [==============================] - 741s 897ms/step - loss: 0.6521 - accuracy: 0.6110 - val_loss: 0.6581 - val_accuracy: 0.5940
Epoch 4/10
825/825 [==============================] - 689s 835ms/step - loss: 0.6466 - accuracy: 0.6176 - val_loss: 0.6574 - val_accuracy: 0.5957
Epoch 5/10
825/825 [==============================] - 734s 889ms/step - loss: 0.6418 - accuracy: 0.6229 - val_loss: 0.6600 - val_accuracy: 0.5956
Epoch 6/10
825/825 [==============================] - 728s 882ms/step - loss: 0.6371 - accuracy: 0.6274 - val_loss: 0.6620 - val_accuracy: 0.5934
Epoch 7/10
825/825 [==============================] - 721s 873ms/step - loss

In [3]:
model_base_id = '327'
version = '3'
model_no = model_base_id
constituent_limits = 0.05
id_constituent = 1
min_vid_constituents = 0.3
epochs = 10
batch_size = 512
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [4]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 2039
Total constituents: 422
Epoch 1/10
3895/3895 [==============================] - 2167s 554ms/step - loss: 0.6732 - accuracy: 0.5884 - val_loss: 0.6601 - val_accuracy: 0.5985
Epoch 2/10
3895/3895 [==============================] - 2236s 574ms/step - loss: 0.6559 - accuracy: 0.6094 - val_loss: 0.6625 - val_accuracy: 0.6004
Epoch 3/10
3895/3895 [==============================] - 2189s 562ms/step - loss: 0.6512 - accuracy: 0.6143 - val_loss: 0.6663 - val_accuracy: 0.6001
Epoch 4/10
3895/3895 [==============================] - 2275s 584ms/step - loss: 0.6470 - accuracy: 0.6182 - val_loss: 0.6710 - val_accuracy: 0.6000
Epoch 5/10
3895/3895 [==============================] - 2222s 570ms/step - loss: 0.6427 - accuracy: 0.6219 - val_loss: 0.6783 - val_accuracy: 0.5884
Epoch 6/10
3895/3895 [==============================] - 2245s 576ms/step - loss: 0.6384 - accuracy: 0.6257 - val_loss: 0.6924 - val_accuracy: 0.5776
Epoch 7/10
3895/3895 [==============================] - 2

In [3]:
model_base_id = '327'
version = '3'
model_no = model_base_id
constituent_limits = 0.025
id_constituent = 1
min_vid_constituents = 0.6
epochs = 10
batch_size = 512
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_2'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [4]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 1416
Total constituents: 422
Epoch 1/10
3521/3521 [==============================] - 1666s 471ms/step - loss: 0.6724 - accuracy: 0.5892 - val_loss: 0.6632 - val_accuracy: 0.5983
Epoch 2/10
3521/3521 [==============================] - 1681s 477ms/step - loss: 0.6542 - accuracy: 0.6113 - val_loss: 0.6621 - val_accuracy: 0.6031
Epoch 3/10
3521/3521 [==============================] - 1694s 481ms/step - loss: 0.6475 - accuracy: 0.6179 - val_loss: 0.6666 - val_accuracy: 0.6011
Epoch 4/10
3521/3521 [==============================] - 1726s 490ms/step - loss: 0.6411 - accuracy: 0.6241 - val_loss: 0.6742 - val_accuracy: 0.5880
Epoch 5/10
3521/3521 [==============================] - 1817s 516ms/step - loss: 0.6345 - accuracy: 0.6299 - val_loss: 0.6834 - val_accuracy: 0.5770
Epoch 6/10
3521/3521 [==============================] - 1795s 510ms/step - loss: 0.6279 - accuracy: 0.6360 - val_loss: 0.6980 - val_accuracy: 0.5693
Epoch 7/10
3521/3521 [==============================] - 1

In [3]:
model_base_id = '327'
version = '5'
model_no = model_base_id
constituent_limits = 0.2
id_constituent = 1
min_vid_constituents = 0.2
epochs = 1
batch_size = 512
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_2'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [ ]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
if type(train_gen) == list:
    for epoch in range(epochs):
        for i, tg in enumerate(train_gen):
            print(f'Epoch {epoch+1}-{i+1}/{epochs}')
            history = model.fit(tg, validation_data=validation_gen, epochs=1, verbose=1)
elif type(train_gen) != list:
    history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 7216
Total constituents: 422
Epoch 1-1/1
5149/5149 [==============================] - 1968s 381ms/step - loss: 0.7509 - accuracy: 0.5217 - val_loss: 0.6954 - val_accuracy: 0.5371
Epoch 1-2/1
5139/5139 [==============================] - 1991s 387ms/step - loss: 0.6865 - accuracy: 0.5517 - val_loss: 0.6803 - val_accuracy: 0.5647
Epoch 1-3/1
5114/5114 [==============================] - 1954s 382ms/step - loss: 0.6756 - accuracy: 0.5742 - val_loss: 0.6718 - val_accuracy: 0.5831
Epoch 1-4/1
5124/5124 [==============================] - 2281s 445ms/step - loss: 0.6686 - accuracy: 0.5872 - val_loss: 0.6673 - val_accuracy: 0.5924
Epoch 1-5/1
5072/5072 [==============================] - 2228s 439ms/step - loss: 0.6651 - accuracy: 0.5940 - val_loss: 0.6645 - val_accuracy: 0.5984
Epoch 1-6/1
   1096/Unknown - 444s 404ms/step - loss: 0.6629 - accuracy: 0.5985

In [3]:
model_base_id = '327'
version = '5'
model_no = model_base_id
constituent_limits = 0.2
id_constituent = 1
min_vid_constituents = 0.2
epochs = 5
batch_size = 1024
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_2'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [4]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
if type(train_gen) == list:
    for epoch in range(epochs):
        for i, tg in enumerate(train_gen):
            print(f'Epoch {epoch+1}-{i+1}/{epochs}')
            history = model.fit(tg, validation_data=validation_gen, epochs=1, verbose=1)
elif type(train_gen) != list:
    history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 7216
Total constituents: 422
Epoch 1-1/5
2596/2596 [==============================] - 2002s 769ms/step - loss: 0.7493 - accuracy: 0.5112 - val_loss: 0.7118 - val_accuracy: 0.5232
Epoch 1-2/5
2541/2541 [==============================] - 1974s 776ms/step - loss: 0.7009 - accuracy: 0.5300 - val_loss: 0.6927 - val_accuracy: 0.5418
Epoch 1-3/5
2569/2569 [==============================] - 1963s 764ms/step - loss: 0.6887 - accuracy: 0.5492 - val_loss: 0.6829 - val_accuracy: 0.5611
Epoch 1-4/5
2530/2530 [==============================] - 1933s 764ms/step - loss: 0.6804 - accuracy: 0.5651 - val_loss: 0.6771 - val_accuracy: 0.5728
Epoch 1-5/5
2543/2543 [==============================] - 1963s 772ms/step - loss: 0.6740 - accuracy: 0.5794 - val_loss: 0.6721 - val_accuracy: 0.5839
Epoch 1-6/5
2606/2606 [==============================] - 1970s 756ms/step - loss: 0.6694 - accuracy: 0.5879 - val_loss: 0.6688 - val_accuracy: 0.5901
Epoch 1-7/5
2592/2592 [============================

In [3]:
model_base_id = '327'
version = '4'
model_no = model_base_id
constituent_limits = 0.5
id_constituent = 1
min_vid_constituents = 0.1
epochs = 10
batch_size = 256
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_2'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [ ]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
if type(train_gen) == list:
    for epoch in range(epochs):
        for i, tg in enumerate(train_gen):
            print(f'Epoch {epoch+1}-{i+1}/{epochs}')
            history = model.fit(tg, validation_data=validation_gen, epochs=1, verbose=1)
elif type(train_gen) != list:
    history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 16419
Total constituents: 422
Epoch 1-1/10
10047/10047 [==============================] - 3721s 369ms/step - loss: 0.6654 - accuracy: 0.5976 - val_loss: 0.6655 - val_accuracy: 0.5941
Epoch 1-2/10
10121/10121 [==============================] - 3557s 351ms/step - loss: 0.6550 - accuracy: 0.6108 - val_loss: 0.6660 - val_accuracy: 0.5963
Epoch 1-3/10
9893/9893 [==============================] - 3348s 338ms/step - loss: 0.6473 - accuracy: 0.6204 - val_loss: 0.6920 - val_accuracy: 0.5760
Epoch 1-4/10
10334/10334 [==============================] - 3471s 336ms/step - loss: 0.6384 - accuracy: 0.6287 - val_loss: 0.7397 - val_accuracy: 0.5681
Epoch 1-5/10
   1899/Unknown - 566s 298ms/step - loss: 0.6314 - accuracy: 0.6360

In [3]:
model_base_id = '327'
version = '3'
model_no = model_base_id
constituent_limits = 0.15
id_constituent = 1
min_vid_constituents = 0.2
epochs = 10
batch_size = 2048
iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_2'
ROOT_PATH='./'
DB_ROOT_PATH='J:\#PROJECT\idx'
db_ver = '8'

shuffle_buffer_size = 1
generator=True

In [4]:
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)

# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)
if type(train_gen) == list:
    for epoch in range(epochs):
        for i, tg in enumerate(train_gen):
            print(f'Epoch {epoch+1}-{i+1}/{epochs}')
            history = model.fit(tg, validation_data=validation_gen, epochs=1, verbose=1)
elif type(train_gen) != list:
    history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
model.save_weights(f'{save_path}/weights/checkpoint')
model.save(f'{save_path}/model')

with open(f'{save_path}/history.json', 'w') as f:
    core.json.dump(history.history, f)

Total constituents: 5428
Total constituents: 422
Epoch 1-1/10
1273/1273 [==============================] - 2120s 2s/step - loss: 0.6760 - accuracy: 0.5828 - val_loss: 0.6578 - val_accuracy: 0.6023
Epoch 1-2/10
1266/1266 [==============================] - 2117s 2s/step - loss: 0.6572 - accuracy: 0.6042 - val_loss: 0.6532 - val_accuracy: 0.6097
Epoch 1-3/10
1309/1309 [==============================] - 2174s 2s/step - loss: 0.6550 - accuracy: 0.6077 - val_loss: 0.6492 - val_accuracy: 0.6146
Epoch 1-4/10
1281/1281 [==============================] - 2149s 2s/step - loss: 0.6538 - accuracy: 0.6094 - val_loss: 0.6499 - val_accuracy: 0.6174
Epoch 1-5/10
1274/1274 [==============================] - 2139s 2s/step - loss: 0.6525 - accuracy: 0.6103 - val_loss: 0.6511 - val_accuracy: 0.6158
Epoch 1-6/10
572/572 [==============================] - 1087s 2s/step - loss: 0.6523 - accuracy: 0.6104 - val_loss: 0.6483 - val_accuracy: 0.6161
Epoch 2-1/10
1273/1273 [==============================] - 2088s 2

In [3]:
model_base_id = '327'
version = '3'
model_no = model_base_id
constituent_limits = 0.5
id_constituent = 1
min_vid_constituents = 0.4
epochs = 5
batch_size = 1024
load_dataset_wsd_ver_switcher = {'1':['1','world data load from test slice'],
                                 '2':['2','world data load from train slice'],}
load_dataset_wsd_ver = '2'
shuffle_buffer_size = 1
generator=True

ROOT_PATH='./'
DB_ROOT_PATH='E:\#PROJECT\idx'
db_ver = '8'

In [ ]:
def epochs_subepochs_progress(folder_list, search_condition):
    '''Return search_results for iter_id 
    
    with format below:
    
    iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_loaddatasetwsdver-{load_dataset_wsd_ver_switcher[load_dataset_wsd_ver][0]}_shufflebuffersize-{shuffle_buffer_size}_generator-{1 if generator else 0}'
    
    with last part is epochs-subepochs progress'''
    search_results = {}
    for fl in folder_list:
        if core.re.search(search_condition, fl):
            progress = fl.split('_')[-1]
            progress_split = tuple(progress.split('-'))
            epochs, sub_epochs = progress_split
            epochs, sub_epochs = int(epochs), int(sub_epochs)
            if epochs not in search_results:
                search_results[epochs] = []
            if sub_epochs not in search_results[epochs]:
                search_results[epochs].append(sub_epochs)
    return search_results

# Resume from file, save every sub-epochs
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)

In [ ]:
# Define suffix-file-name (as complete as possible, contains all required params to reproduce)
# iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_loaddatasetwsdver-{load_dataset_wsd_ver_switcher[load_dataset_wsd_ver][0]}_shufflebuffersize-{shuffle_buffer_size}_generator-{1 if generator else 0}'
iter_id = f'{model_no}-{version}_cl-{constituent_limits}_idc-{id_constituent}_mid-{min_vid_constituents}_ep-{epochs}_bz-{batch_size}_ldtv-{load_dataset_wsd_ver_switcher[load_dataset_wsd_ver][0]}_sbuffs-{shuffle_buffer_size}_gen-{1 if generator else 0}'

# Search for eligibility in selected folder
preloaded_folder = core.os.path.join(ROOT_PATH, 'models/preloaded')
save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
folder_list = core.os.listdir(preloaded_folder)

# split, select last entry, split again into epochs-subepochs -> dictionary entry
search_results = epochs_subepochs_progress(folder_list, iter_id)

# Load default model if current entries aren't exists.
# Load model backbone
model = core.model_switcher_preloaded(model_no, version=version)

if type(train_gen) == list:
    for epoch in range(epochs):
        for i, tg in enumerate(train_gen):
            complete_iter_id = f'{iter_id}_{epoch}-{i}'
            save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{complete_iter_id}')
            # Check epochs:
            # Check if current complete file name already in folder
            # If true, load model from file, continue to next loop
            if epoch in search_results:
                if i in search_results[epoch]:
                    print(f'Epoch {epoch}-{i}/{epochs-1}: load existing model and continue...')
                    model = core.tf.keras.models.load_model(f'{save_path}/model')
                    
                    # The loops are continued until the condition is false -> training begin using 
                    # last model in progress
                    continue
                elif i not in search_results[epoch]:
                    print(f'Epoch {epoch}-{i}/{epochs-1}: processing...')
            elif epoch not in search_results:
                print(f'Epoch {epoch}-{i}/{epochs-1}: processing...')
            
            # Start retrain model if latest progress has been loaded.
            history = model.fit(tg, validation_data=validation_gen, epochs=1, verbose=1)
            
            # save model after training, only and only if the model is retrained
            model.save_weights(f'{save_path}/weights/checkpoint')
            model.save(f'{save_path}/model')
            
            # Save history.json if the model is retrained
            with open(f'{save_path}/history.json', 'w') as f:
                core.json.dump(history.history, f)
                
            # The model not reloaded from file if the model is retrained
elif type(train_gen) != list:
    history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)
    with open(f'{save_path}/history.json', 'w') as f:
        core.json.dump(history.history, f)
        
# Finish.

In [3]:
model_base_id = '327'
version = '9'
model_no = model_base_id
constituent_limits = 0.35
id_constituent = 1
min_vid_constituents = 0.8
epochs = 40
batch_size = 2048
load_dataset_wsd_ver_switcher = {'1':['1','world data load from test slice'],
                                 '2':['2','world data load from train slice'],
                                 '3':['3','validation data from test set'],}
load_dataset_wsd_ver = '3'
shuffle_buffer_size = 1
generator = True
hotstart = True
# hotstart_from = '327-7_cl-0.005_idc-1_mid-0.3_ep-40_bz-1792_ldtv-3_sbuffs-1_gen-1_3-0'
hotstart_from = '327-9_cl-0.001_idc-0.25_mid-0.9_ep-40_bz-512_ldtv-3_sbuffs-1_gen-1_15-0' # 0.6690 / 0.5891
hotstart_from = '327-9_cl-0.005_idc-1_mid-0.9_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_28-0' # 0.6459 / 0.6173
hotstart_from = '327-9_cl-0.1_idc-1_mid-0.9_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_7-5' # 0.6371 / 0.6281
hotstart_from = '327-9_cl-0.15_idc-1_mid-0.9_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_7-1' # 0.6298 / 0.6349
hotstart_from = '327-9_cl-0.2_idc-1_mid-0.9_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_6-3' # 0.6232 / 0.6408
hotstart_from = '327-9_cl-0.25_idc-1_mid-0.8_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_3-5' # 0.6212 / 0.6433
hotstart_from = '327-9_cl-0.3_idc-1_mid-0.8_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_3-4' # 0.6185 / 0.6465

# hotstart_from = '327-10_cl-0.001_idc-0.25_mid-0.8_ep-40_bz-512_ldtv-3_sbuffs-1_gen-1_10-0' # 0.6703 / 0.5885
# hotstart_from = '327-10_cl-0.005_idc-1_mid-0.8_ep-40_bz-512_ldtv-3_sbuffs-1_gen-1_7-0' # 0.6574 / 0.6065
# hotstart_from = '327-10_cl-0.1_idc-1_mid-0.5_ep-40_bz-512_ldtv-3_sbuffs-1_gen-1_1-4' # 0.6535 / 0.6123
distributed = True

ROOT_PATH='./'
DB_ROOT_PATH='E:\#PROJECT\idx'
db_ver = '8'

In [4]:
def epochs_subepochs_progress(folder_list, search_condition):
    '''Return search_results for iter_id 
    
    with format below:
    
    iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_loaddatasetwsdver-{load_dataset_wsd_ver_switcher[load_dataset_wsd_ver][0]}_shufflebuffersize-{shuffle_buffer_size}_generator-{1 if generator else 0}'
    
    with last part is epochs-subepochs progress'''
    search_results = {}
    for fl in folder_list:
        if core.re.search(search_condition, fl):
            progress = fl.split('_')[-1]
            progress_split = tuple(progress.split('-'))
            epochs, sub_epochs = progress_split
            epochs, sub_epochs = int(epochs), int(sub_epochs)
            if epochs not in search_results:
                search_results[epochs] = []
            if sub_epochs not in search_results[epochs]:
                search_results[epochs].append(sub_epochs)
    return search_results

# Resume from file, save every sub-epochs
# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH=DB_ROOT_PATH, db_ver=db_ver, constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no, min_vid_constituents=min_vid_constituents)

Total constituents: 19079
Total constituents: 422


In [ ]:
# Define suffix-file-name (as complete as possible, contains all required params to reproduce)
# iter_id = f'model-{model_no}-{version}_constituentlimits-{constituent_limits}_idconstituent-{id_constituent}_minvidconstituents-{min_vid_constituents}_epochs-{epochs}_batchsize-{batch_size}_loaddatasetwsdver-{load_dataset_wsd_ver_switcher[load_dataset_wsd_ver][0]}_shufflebuffersize-{shuffle_buffer_size}_generator-{1 if generator else 0}'
iter_id = f'{model_no}-{version}_cl-{constituent_limits}_idc-{id_constituent}_mid-{min_vid_constituents}_ep-{epochs}_bz-{batch_size}_ldtv-{load_dataset_wsd_ver_switcher[load_dataset_wsd_ver][0]}_sbuffs-{shuffle_buffer_size}_gen-{1 if generator else 0}'

# Search for eligibility in selected folder
preloaded_folder = core.os.path.join(ROOT_PATH, 'models/preloaded')
save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{iter_id}/')
folder_list = core.os.listdir(preloaded_folder)

# split, select last entry, split again into epochs-subepochs -> dictionary entry
search_results = epochs_subepochs_progress(folder_list, iter_id)

if distributed:
    core.tf.debugging.set_log_device_placement(True)
    gpus = core.tf.config.list_logical_devices('GPU')
    strategy = core.tf.distribute.MirroredStrategy(gpus)
    with strategy.scope():
        # Load default model if current entries aren't exists.
        # Load model backbone
        model = core.model_switcher_preloaded(model_no, version=version)
        
        if hotstart:
            hotstart_save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{hotstart_from}')
            model = core.tf.keras.models.load_model(f'{hotstart_save_path}/model')

        if type(train_gen) == list:
            for epoch in range(epochs):
                for i, tg in enumerate(train_gen):
                    complete_iter_id = f'{iter_id}_{epoch}-{i}'
                    save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{complete_iter_id}')
                    # Check epochs:
                    # Check if current complete file name already in folder
                    # If true, load model from file, continue to next loop
                    if epoch in search_results:
                        if i in search_results[epoch]:
                            print(f'Epoch {epoch}-{i}/{epochs-1}: load existing model and continue...')
                            model = core.tf.keras.models.load_model(f'{save_path}/model')

                            # The loops are continued until the condition is false -> training begin using 
                            # last model in progress
                            continue
                        elif i not in search_results[epoch]:
                            print(f'Epoch {epoch}-{i}/{epochs-1}: processing...')
                    elif epoch not in search_results:
                        print(f'Epoch {epoch}-{i}/{epochs-1}: processing...')

                    # Start retrain model if latest progress has been loaded.
                    history = model.fit(tg, validation_data=validation_gen, epochs=1, verbose=1)

                    # save model after training, only and only if the model is retrained
                    model.save_weights(f'{save_path}/weights/checkpoint')
                    model.save(f'{save_path}/model')

                    # Save history.json if the model is retrained
                    with open(f'{save_path}/history.json', 'w') as f:
                        core.json.dump(history.history, f)

                    # The model not reloaded from file if the model is retrained
        elif type(train_gen) != list:
            history = model.fit(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)
            with open(f'{save_path}/history.json', 'w') as f:
                core.json.dump(history.history, f)
                
elif not distributed:
    print('Please modify some code to run in not distributed mode.')
        
# Finish.

Epoch 0-0/39: processing...
1208/1208 [==============================] - 1675s 1s/step - loss: 0.6362 - accuracy: 0.6263 - val_loss: 0.6184 - val_accuracy: 0.6459


Epoch 0-1/39: processing...
1133/1133 [==============================] - 1564s 1s/step - loss: 0.6390 - accuracy: 0.6232 - val_loss: 0.6176 - val_accuracy: 0.6467


Epoch 0-2/39: processing...
1193/1193 [==============================] - 1631s 1s/step - loss: 0.6375 - accuracy: 0.6239 - val_loss: 0.6187 - val_accuracy: 0.6462


Epoch 0-3/39: processing...
1140/1140 [==============================] - 1577s 1s/step - loss: 0.6367 - accuracy: 0.6253 - val_loss: 0.6182 - val_accuracy: 0.6465


Epoch 0-4/39: processing...
1163/1163 [==============================] - 1602s 1s/step - loss: 0.6373 - accuracy: 0.6244 - val_loss: 0.6185 - val_accuracy: 0.6469


...

Epoch 0-10/39: processing...
1154/1154 [==============================] - 1878s 2s/step - loss: 0.6378 - accuracy: 0.6243 - val_loss: 0.6187 - val_accuracy: 0.6464


Epoch 0-11/39: processing...
1137/1137 [==============================] - 2332s 2s/step - loss: 0.6386 - accuracy: 0.6240 - val_loss: 0.6182 - val_accuracy: 0.6469


Epoch 0-12/39: processing...
1173/1173 [==============================] - 1733s 1s/step - loss: 0.6384 - accuracy: 0.6241 - val_loss: 0.6179 - val_accuracy: 0.6472


Epoch 0-13/39: processing...
1145/1145 [==============================] - 1721s 2s/step - loss: 0.6395 - accuracy: 0.6227 - val_loss: 0.6182 - val_accuracy: 0.6466


Epoch 0-14/39: processing...
    173/Unknown - 255s 1s/step - loss: 0.6345 - accuracy: 0.6279